# UoA Copcsi 765 Assignemnt 1
Shuo Mao 437681250

In [1]:
# loading data
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.utils import resample

file_path = 'website-phishing.csv'
web = pd.read_csv(file_path)
 
bcp = pd.read_csv("BCP.csv")
arrhy = pd.read_csv("arrhythmia.csv")

In [ ]:
web.head(3)

In [ ]:
bcp.head(3)

In [ ]:
arrhy.head(3)

By manually brife view the row data of each one, that I discover in the dataset of arrhythmia have missing value repersenting by question mark `?`, thus I'll read through all data and replace `?` by `Null` value, so the machineism can process feature.

In [ ]:
# Replace '?' with np.nan for the entire DataFrame
arrhy.replace('?', np.nan, inplace=True)
# Convert all columns to numeric, errors='coerce' will convert errors (non-convertible values) to NaN
data = arrhy.apply(pd.to_numeric, errors='coerce')

# Replace NaN values with the mean of their respective columns for the entire DataFrame
data.fillna(data.mean(), inplace=True)
#generate output csv file for following steps
data.to_csv('new_arrhy.csv', index=False, header=True)

# 1: Decision Stump
A decision stump is a decision tree with a depth of 1. It splits the dataset using the best feature based on a criterion like Gini impurity or information gain.

In [ ]:
import numpy as np

class DecisionStump:
    def __init__(self):
        # Initialize the decision stump with default values
        self.threshold = None  # The value to decide if a data point goes left or right
        self.feature_index = None  # The index of the feature to split on
        self.left_value = None  # The most common class on the left side of the stump
        self.right_value = None  # The most common class on the right side of the stump
    
    
    def fit(self, X, y):
        best_info_gain = -1  # Initialize the best information gain to a value that will be updated
        # Iterate over all features in the dataset
        for feature_index in range(X.shape[1]): # in this case feature_index repersending columns in row dataset
            thresholds = np.unique(X[:, feature_index])  # Get all unique values of the feature as potential thresholds
            for threshold in thresholds:
                # Calculate the information gain of splitting on this feature at this threshold
                info_gain = self.information_gain(y, X[:, feature_index], threshold)
                if info_gain > best_info_gain:
                    # If this split provides a better information gain, update the stump's parameters
                    best_info_gain = info_gain
                    self.threshold = threshold
                    self.feature_index = feature_index
        
        # After finding the best split, determine the output value (class) for each side of the split
        # This uses the mean of the target values, which implicitly handles binary classification (0 and 1)
        self.left_value = np.round(np.mean(y[X[:, self.feature_index] <= self.threshold]))
        self.right_value = np.round(np.mean(y[X[:, self.feature_index] > self.threshold]))
        
        # Determine the value for the left and right child nodes
        self.left_value = np.round(np.mean(y[X[:, self.feature_index] <= self.threshold]))
        self.right_value = np.round(np.mean(y[X[:, self.feature_index] > self.threshold]))
        
    def predict(self, X):
        # Generate predictions based on the threshold and feature index
        predictions = np.where(X[:, self.feature_index] <= self.threshold, self.left_value, self.right_value)
        return predictions

    
    def information_gain(self, y, feature_column, threshold):
        parent_entropy = self.entropy(y)  # Calculate the entropy before the split
        
        # Create masks to split the data based on the threshold
        left_child_index = feature_column <= threshold
        right_child_index = feature_column > threshold
        
        # Ensure both sides of the split contain samples
        if np.any(left_child_index) and np.any(right_child_index):
            n = len(y)
            n_left = np.sum(left_child_index)  # Number of samples in the left split
            n_right = np.sum(right_child_index)  # Number of samples in the right split
            # Weighted average of child entropies
            child_entropy = (n_left / n) * self.entropy(y[left_child_index]) + (n_right / n) * self.entropy(y[right_child_index])
            info_gain = parent_entropy - child_entropy  # Information gain is reduction in entropy
            return info_gain
        else:
            return 0

    
    def entropy(self, y):
        if np.any(y < 0):  # Handle datasets where the negative class is labeled as -1
            y = np.where(y == -1, 0, 1)
        proportions = np.bincount(y) / len(y)
        return -np.sum([p * np.log2(p) for p in proportions if p > 0])



In [ ]:
# Assume the dataset has been loaded into a DataFrame named data
# and the necessary preprocessing has been done.
# Splitting the dataset into features (X) and target (y)
stump = DecisionStump()


X = web.iloc[:, :-1].values# Selecting all expect y(class)
y = web.iloc[:, -1].values# Selecting Class

# train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=43)


#fit the decision stump

stump.fit(X_train, y_train)

# Use the stump to make predictions
predictions = stump.predict(X_test)
# Optionally, evaluate the stump's performance
accuracy = np.mean(predictions == y_test)

print(f'Accuracy of the decision stump in website-phisihing dataset: {accuracy:.4f}')




In [ ]:

X = bcp.iloc[:, :-1].values
y = bcp.iloc[:, -1].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=78)
stump.fit(X_train, y_train)
# Use the stump to make predictions
predictions = stump.predict(X_test)
# Optionally, evaluate the stump's performance
accuracy = np.mean(predictions == y_test)

print(f'Accuracy of the decision stump in BCP dataset: {accuracy:.4f}')



In [ ]:
arrhy_aft = pd.read_csv("new_arrhy.csv")

X= arrhy_aft.iloc[:,:-1].values
y = arrhy_aft.iloc[:,-1].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
stump.fit(X_train, y_train)
# Use the stump to make predictions
predictions = stump.predict(X_test)
# Optionally, evaluate the stump's performance
accuracy = np.mean(predictions == y_test)

print(f'Accuracy of the decision stump in arrhythmia dataset: {accuracy:.4f}')


After applying the Decision Stump method to each raw dataset, it was discovered that the Decision Stump works effectively for basic binary classification tasks. For example, in the website-phishing dataset, which has just two possible outcomes `(-1, 1)`, and the BCP dataset `(with outcomes 2,4)`, the accuracy rates were pretty high, about `0.89`.This highlights the effectiveness of Decision Stumps in binary classification scenarios. In contrast, the Decision Stump performs substantially worse in non-binary classification cases, such as the arrhythmia dataset, which has `13` alternative outcomes. In this example, the accuracy dropped to a meager `0.022`, indicating a `2%` probability of making a right forecast. This striking disparity demonstrates Decision Stumps' limits in handling increasingly complicated, non-binary categorization tasks."


# 2: Unpruned Decision Tree
An unpruned decision tree grows until all leaves are pure or until every leaf contains a minimum number of samples.

In [2]:
class Node():
    def __init__(self, feature_index=None, threshold=None, left=None, right=None, info_gain=None, value=None):
        # Node class to store information about each point in the decision tree.
        self.feature_index = feature_index  # Index of the feature used for splitting
        self.threshold = threshold  # Threshold value for the split at this node
        self.left = left  # Reference to the left child node
        self.right = right  # Reference to the right child node
        self.info_gain = info_gain  # Information gain from the split at this node
        self.value = value  # The class value if this is a leaf node, else None



class DecisionTreeClassifier():
    def __init__(self, min_samples_split=2, max_depth=2):
        # Initialize the decision tree classifier with stopping conditions.
        self.root = None  # Root node of the tree, initially None
        self.min_samples_split = min_samples_split  # Minimum number of samples required to split a node
        self.max_depth = max_depth  # Maximum depth of the tree

        
    def build_tree(self, dataset, curr_depth=0):
        # Recursively builds the decision tree from the given dataset.
        X, Y = dataset[:,:-1], dataset[:,-1]  # Split dataset into features (X) and target (Y)
        num_samples, num_features = np.shape(X)
        
        # Base case: stop splitting if minimum samples or maximum depth criteria are met
        if num_samples >= self.min_samples_split and curr_depth <= self.max_depth:
            best_split = self.get_best_split(dataset, num_samples, num_features)
            if best_split["info_gain"] > 0:  # Ensure positive information gain for splitting
                # Recursively build left and right subtrees
                left_subtree = self.build_tree(best_split["dataset_left"], curr_depth+1)
                right_subtree = self.build_tree(best_split["dataset_right"], curr_depth+1)
                return Node(best_split["feature_index"], best_split["threshold"], left_subtree, right_subtree, best_split["info_gain"])
        
        return Node(value=self.calculate_leaf_value(Y))  # Leaf node with class prediction

    
    def get_best_split(self, dataset, num_samples, num_features):
        """ This method identifies the optimal feature and threshold for splitting the dataset into two child nodes. It aims to maximize the information gain from the split."""
        # Initialize the dictionary to store the best split found during the process
        best_split = {}
        # Initialize the maximum information gain to a very low number
        max_info_gain = -float("inf")
         # Iterate over all features in the dataset
        for feature_index in range(num_features):
            # Extract all values for the current feature
            feature_values = dataset[:, feature_index]
            # Identify all unique values of the feature to consider as potential thresholds
            possible_thresholds = np.unique(feature_values)
             # Test each threshold to find the best split for the current feature
            for threshold in possible_thresholds:
                # Split the dataset into two parts: one part where the feature values are less than or equal to the threshold
                 # and another part where the feature values are greater than the threshold
                dataset_left, dataset_right = self.split(dataset, feature_index, threshold)
                # Ensure both left and right splits contain data
                if len(dataset_left) > 0 and len(dataset_right) > 0:
                    # Extract the target values for the whole dataset, and each of the splits
                    y, left_y, right_y = dataset[:, -1], dataset_left[:, -1], dataset_right[:, -1]
                    # Calculate the information gain achieved by the current split
                    curr_info_gain = self.information_gain(y, left_y, right_y, "gini")
                    # If the information gain from the current split is higher than the maximum found so far,
                    # update the best split to this split
                    if curr_info_gain > max_info_gain:
                        best_split["feature_index"] = feature_index
                        best_split["threshold"] = threshold
                        best_split["dataset_left"] = dataset_left
                        best_split["dataset_right"] = dataset_right
                        best_split["info_gain"] = curr_info_gain
                        max_info_gain = curr_info_gain
        return best_split

    
    def split(self, dataset, feature_index, threshold):
        # Creates an empty array for the left dataset. This will hold all rows
        # where the value in the specified feature column is less than or equal
        # to the threshold.
        dataset_left = np.array([row for row in dataset if row[feature_index] <= threshold]) 
        # Similarly, creates an empty array for the right dataset. This will
        # contain all rows where the value in the specified feature column is
        # greater than the threshold.
        dataset_right = np.array([row for row in dataset if row[feature_index] > threshold])
        # Returns the two subsets. These will be used as the new datasets for
        # the subsequent recursive calls to build the left and right subtrees,
        # respectively.
        return dataset_left, dataset_right


    def information_gain(self, parent, l_child, r_child, mode="entropy"):  
        # Calculate the proportion of samples that go to the left and right child nodes.
        weight_l = len(l_child) / len(parent)
        weight_r = len(r_child) / len(parent)
        # If the mode is set to "gini", use the Gini impurity measure to calculate information gain.
        if mode == "gini":
            # Information gain is the impurity of the parent node minus the weighted sum of the impurity
            # of the left and right child nodes.
            gain = self.gini_index(parent) - (weight_l * self.gini_index(l_child) + weight_r * self.gini_index(r_child))
        else:
            # If the mode is not "gini" (default is "entropy"), use the entropy measure to calculate information gain.
            gain = self.entropy(parent) - (weight_l * self.entropy(l_child) + weight_r * self.entropy(r_child))
        
        return gain

    
    def entropy(self, y):
        # Identify the unique class labels in the dataset.
        class_labels = np.unique(y)
        # Initialize entropy to zero. Entropy will accumulate the disorder measure for each class.
        entropy = 0
        # Iterate over each class label. 
        for cls in class_labels:
            # Calculate the proportion (p_cls) of instances in the dataset belonging to the current class.
            p_cls = len(y[y == cls]) / len(y)
            # Update the entropy using the entropy formula part associated with this class.
            # The formula for entropy of a class is: -p * log2(p), where p is the class proportion.
            entropy += -p_cls * np.log2(p_cls)
        # Return the total entropy, which represents the disorder within the entire dataset.
        return entropy

    
    def gini_index(self, y):      
        # Identify the unique class labels in the dataset.
        class_labels = np.unique(y)
        # Initialize gini to zero. This will accumulate the squared proportion of each class.
        gini = 0
        # Iterate over each class label.
        for cls in class_labels:
            # Calculate the proportion (p_cls) of instances in the dataset belonging to the current class.
            p_cls = len(y[y == cls]) / len(y)
            # Accumulate the square of the proportion of the current class to the gini score.
            gini += p_cls**2
        # Gini impurity is calculated as 1 minus the accumulated score.
        # This transformation ensures that a higher value indicates higher impurity.
        return 1 - gini

        
    def calculate_leaf_value(self, Y):
        # Convert Y to a list if it's not already one. This ensures compatibility with the 'max' function and 'count' method.
        Y = list(Y)
        # Use the 'max' function with a key that counts the occurrences of each class label in Y.
        # This returns the class label that appears most frequently in Y.
        return max(Y, key=Y.count)

    
    def print_tree(self, tree=None, indent=" "):
        ''' function to print the tree '''     
        if not tree:
            tree = self.root
        if tree.value is not None:
            print(tree.value)
        else:
            print("X_"+str(tree.feature_index), "<=", tree.threshold, "?", tree.info_gain)
            print("%sleft:" % (indent), end="")
            self.print_tree(tree.left, indent + indent)
            print("%sright:" % (indent), end="")
            self.print_tree(tree.right, indent + indent)
    
    def fit(self, X, Y):
        # Ensures that Y is in the correct shape.
        # If Y is a 1-dimensional array (a common format for labels), it reshapes it to a 2-dimensional array with a single column.
        # This uniform shape is necessary for concatenating Y with X.
        if Y.ndim == 1:
            Y = Y.reshape(-1, 1)
        # This combined dataset format is convenient for the subsequent operations of splitting and building the tree.
        dataset = np.concatenate((X, Y), axis=1)
        # Initiates the recursive process of building the decision tree.
        # The build_tree method will be called with the entire dataset,
        # and it will recursively split the dataset on the best features and thresholds,
        # until it meets the stopping criteria to create leaf nodes.
        self.root = self.build_tree(dataset)

    
    def predict(self, X):
        # This line iterates over each example in the dataset X. For each example (x),
        # it calls the make_prediction method, starting from the root of the tree.
        # The result is a list of predictions corresponding to each example in X.
        predictions = [self.make_prediction(x, self.root) for x in X]
        return predictions

    
    def make_prediction(self, x, tree):
        # Checks if the current node (tree) is a leaf node by verifying if it has a non-None value.
        # Leaf nodes in a decision tree do not have further splits and contain a class value (prediction).
        if tree.value is not None:
            return tree.value  # If it's a leaf node, return the value (class label) of the leaf node.
        # Retrieves the feature value from the instance x for the feature at index tree.feature_index.
        # This value is used to decide whether to move left or right down the tree.
        feature_val = x[tree.feature_index]
        # If the feature value is less than or equal to the threshold of the current node,
        # the method recursively calls itself to move to the left child of the current node.
        if feature_val <= tree.threshold:
            return self.make_prediction(x, tree.left)
        else:
            # If the feature value is greater than the threshold of the current node,
            # the method recursively calls itself to move to the right child of the current node.
            return self.make_prediction(x, tree.right)


In [14]:
# loading website-phishing 
file_path = 'website-phishing.csv'
data = pd.read_csv(file_path, skiprows=1, header=None)

In [15]:

X = data.iloc[:, :-1].values
y = data.iloc[:, -1].values.reshape(-1, 1)  

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=41)

classifier = DecisionTreeClassifier(min_samples_split=2, max_depth=5)
classifier.fit(X_train, y_train)

y_pred = classifier.predict(X_test) 
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

Accuracy: 0.9249208502939846


In this investigation, I used `min_samples_split = 2` for our classification issue, which corresponds to the binary character of our results (true or false). This setting is often beneficial in such situations. For `max_depth`, I chose a depth of 5. The objective for this decision is to create a balance between model complexity and computing efficiency while avoiding overfitting. A tree that is overly deep may result in overfitting, which occurs when the model learns noise from the training data rather than the real signal, as well as considerably increasing computation time. In contrast, a shallow tree, while quicker, may not capture the underlying patterns well, resulting in underfitting.

I ran tests with `max_depth` values of 2, 5, 8, and 12. A depth of two was insufficient to capture the intricacies of the data. While a depth of 8 produced more accurate results, the longer calculating time was a considerable disadvantage. A depth of 12 resulted in overfitting concerns, indicating that the model was too complicated for our data. Thus, a depth of 5 emerged as the best option, giving a reasonable balance of model accuracy and efficiency, ensuring the model is sophisticated enough to discover meaningful patterns without spending too much time on computations or fitting to noise."

In [ ]:
#Ploting tree structure of web data
classifier.print_tree()

In [5]:
# loading BCP
file_path = 'BCP.csv'
data = pd.read_csv(file_path, skiprows=1, header=None)

In [10]:
X = data.iloc[:, :-1].values
y = data.iloc[:, -1].values.reshape(-1, 1)  

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=41)

classifier = DecisionTreeClassifier(min_samples_split=2, max_depth=5)
classifier.fit(X_train, y_train)

y_pred = classifier.predict(X_test) 
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

Accuracy: 0.9197080291970803


In [ ]:
# ploting tree 
classifier.print_tree()

In [12]:
# loading arrhythmia
file_path = 'new_arrhy.csv'#Notice do not loading original dataset program will crash
data = pd.read_csv(file_path, skiprows=1, header=None)

In [13]:
X = data.iloc[:, :-1].values
y = data.iloc[:, -1].values.reshape(-1, 1)  

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=41)

classifier = DecisionTreeClassifier(min_samples_split=2, max_depth=5)
classifier.fit(X_train, y_train)

y_pred = classifier.predict(X_test) 
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

Accuracy: 0.6153846153846154


In [ ]:
# Plotting tree
classifier.print_tree()

"After implementing the Decision Tree algorithm on each row of data, observations indicate a distinct performance pattern across different types of classification tasks. In simple binary classification scenarios, the improvement in performance is marginal. However, a significant enhancement is observed in non-binary classification tasks, where the accuracy rate dramatically increased from `0.022` to `0.670`. This improvement aligns with expectations, highlighting the Decision Tree's capability to handle complex classification problems with multiple classes more effectively than simpler, binary classification tasks."

# 3: Pruned Decision Tree
Pruning can be done using several strategies like reduced error pruning (post-pruning) or setting a maximum depth (pre-pruning). 

In [ ]:
class Node():
    def __init__(self, feature_index=None, threshold=None, left=None, right=None, info_gain=None, value=None):
        self.feature_index = feature_index
        self.threshold = threshold
        self.left = left
        self.right = right
        self.info_gain = info_gain
        self.value = value

class PurningDecisionTreeClassifier():
    def __init__(self, min_samples_split=2, max_depth=2):
        self.root = None
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.label_mapping = {}
        self.most_common_class = None

    def fit(self, X, Y):
        # Map labels to non-negative integers if necessary
        self.label_mapping, mapped_Y = self._map_labels(Y)
        # Calculate and set the most common class (using the mapped labels)
        self.most_common_class = np.bincount(mapped_Y.flatten()).argmax()
        dataset = np.concatenate((X, mapped_Y), axis=1)
        self.root = self.build_tree(dataset)

    
    def _evaluate_error(self, X, Y):
        predictions = self.predict(X)
        error = np.mean(predictions != Y.flatten())  # Flattening Y to ensure shape compatibility
        return error

    def predict(self, X):
        predictions = [self._make_prediction(x, self.root) for x in X]
        # Reverse map labels to original
        return np.array([self.label_mapping.get(pred, pred) for pred in predictions])

    def make_prediction(self, x, tree):
        # Checks if the current node (tree) is a leaf node by verifying if it has a non-None value.
        # Leaf nodes in a decision tree do not have further splits and contain a class value (prediction).
        if tree.value is not None:
            return tree.value  # If it's a leaf node, return the value (class label) of the leaf node.
        # Retrieves the feature value from the instance x for the feature at index tree.feature_index.
        # This value is used to decide whether to move left or right down the tree.
        feature_val = x[tree.feature_index]
        # If the feature value is less than or equal to the threshold of the current node,
        # the method recursively calls itself to move to the left child of the current node.
        if feature_val <= tree.threshold:
            return self.make_prediction(x, tree.left)
        else:
            # If the feature value is greater than the threshold of the current node,
            # the method recursively calls itself to move to the right child of the current node.
            return self.make_prediction(x, tree.right)

    def build_tree(self, dataset, curr_depth=0):
        # Recursively builds the decision tree from the given dataset.
        X, Y = dataset[:,:-1], dataset[:,-1]  # Split dataset into features (X) and target (Y)
        num_samples, num_features = np.shape(X)
        
        # Base case: stop splitting if minimum samples or maximum depth criteria are met
        if num_samples >= self.min_samples_split and curr_depth <= self.max_depth:
            best_split = self.get_best_split(dataset, num_samples, num_features)
            if best_split["info_gain"] > 0:  # Ensure positive information gain for splitting
                # Recursively build left and right subtrees
                left_subtree = self.build_tree(best_split["dataset_left"], curr_depth+1)
                right_subtree = self.build_tree(best_split["dataset_right"], curr_depth+1)
                return Node(best_split["feature_index"], best_split["threshold"], left_subtree, right_subtree, best_split["info_gain"])
        
        return Node(value=self.calculate_leaf_value(Y))  # Leaf node with class prediction

    
    def get_best_split(self, dataset, num_samples, num_features):
        """ This method identifies the optimal feature and threshold for splitting the dataset into two child nodes. It aims to maximize the information gain from the split."""
        # Initialize the dictionary to store the best split found during the process
        best_split = {}
        # Initialize the maximum information gain to a very low number
        max_info_gain = -float("inf")
         # Iterate over all features in the dataset
        for feature_index in range(num_features):
            # Extract all values for the current feature
            feature_values = dataset[:, feature_index]
            # Identify all unique values of the feature to consider as potential thresholds
            possible_thresholds = np.unique(feature_values)
             # Test each threshold to find the best split for the current feature
            for threshold in possible_thresholds:
                # Split the dataset into two parts: one part where the feature values are less than or equal to the threshold
                 # and another part where the feature values are greater than the threshold
                dataset_left, dataset_right = self.split(dataset, feature_index, threshold)
                # Ensure both left and right splits contain data
                if len(dataset_left) > 0 and len(dataset_right) > 0:
                    # Extract the target values for the whole dataset, and each of the splits
                    y, left_y, right_y = dataset[:, -1], dataset_left[:, -1], dataset_right[:, -1]
                    # Calculate the information gain achieved by the current split
                    curr_info_gain = self.information_gain(y, left_y, right_y, "gini")
                    # If the information gain from the current split is higher than the maximum found so far,
                    # update the best split to this split
                    if curr_info_gain > max_info_gain:
                        best_split["feature_index"] = feature_index
                        best_split["threshold"] = threshold
                        best_split["dataset_left"] = dataset_left
                        best_split["dataset_right"] = dataset_right
                        best_split["info_gain"] = curr_info_gain
                        max_info_gain = curr_info_gain
        return best_split

    
    def split(self, dataset, feature_index, threshold):
        # Creates an empty array for the left dataset. This will hold all rows
        # where the value in the specified feature column is less than or equal
        # to the threshold.
        dataset_left = np.array([row for row in dataset if row[feature_index] <= threshold]) 
        # Similarly, creates an empty array for the right dataset. This will
        # contain all rows where the value in the specified feature column is
        # greater than the threshold.
        dataset_right = np.array([row for row in dataset if row[feature_index] > threshold])
        # Returns the two subsets. These will be used as the new datasets for
        # the subsequent recursive calls to build the left and right subtrees,
        # respectively.
        return dataset_left, dataset_right


    def information_gain(self, parent, l_child, r_child, mode="entropy"):  
        # Calculate the proportion of samples that go to the left and right child nodes.
        weight_l = len(l_child) / len(parent)
        weight_r = len(r_child) / len(parent)
        # If the mode is set to "gini", use the Gini impurity measure to calculate information gain.
        if mode == "gini":
            # Information gain is the impurity of the parent node minus the weighted sum of the impurity
            # of the left and right child nodes.
            gain = self.gini_index(parent) - (weight_l * self.gini_index(l_child) + weight_r * self.gini_index(r_child))
        else:
            # If the mode is not "gini" (default is "entropy"), use the entropy measure to calculate information gain.
            gain = self.entropy(parent) - (weight_l * self.entropy(l_child) + weight_r * self.entropy(r_child))
        
        return gain

    
    def entropy(self, y):
        # Identify the unique class labels in the dataset.
        class_labels = np.unique(y)
        # Initialize entropy to zero. Entropy will accumulate the disorder measure for each class.
        entropy = 0
        # Iterate over each class label. 
        for cls in class_labels:
            # Calculate the proportion (p_cls) of instances in the dataset belonging to the current class.
            p_cls = len(y[y == cls]) / len(y)
            # Update the entropy using the entropy formula part associated with this class.
            # The formula for entropy of a class is: -p * log2(p), where p is the class proportion.
            entropy += -p_cls * np.log2(p_cls)
        # Return the total entropy, which represents the disorder within the entire dataset.
        return entropy

    
    def gini_index(self, y):      
        # Identify the unique class labels in the dataset.
        class_labels = np.unique(y)
        # Initialize gini to zero. This will accumulate the squared proportion of each class.
        gini = 0
        # Iterate over each class label.
        for cls in class_labels:
            # Calculate the proportion (p_cls) of instances in the dataset belonging to the current class.
            p_cls = len(y[y == cls]) / len(y)
            # Accumulate the square of the proportion of the current class to the gini score.
            gini += p_cls**2
        # Gini impurity is calculated as 1 minus the accumulated score.
        # This transformation ensures that a higher value indicates higher impurity.
        return 1 - gini

        
    def calculate_leaf_value(self, Y):
        ''' function to compute leaf node '''
        if len(Y) == 0:  # Check if Y is empty
            # Return the most common class of the entire dataset
            return self.most_common_class
        # Convert Y to a flat list of labels if it's not already
        flat_Y = Y.flatten()
        # Calculate the most common class label in Y
        return np.bincount(flat_Y).argmax()


    
    def print_tree(self, tree=None, indent=" "):
        ''' function to print the tree '''     
        if not tree:
            tree = self.root
        if tree.value is not None:
            print(tree.value)
        else:
            print("X_"+str(tree.feature_index), "<=", tree.threshold, "?", tree.info_gain)
            print("%sleft:" % (indent), end="")
            self.print_tree(tree.left, indent + indent)
            print("%sright:" % (indent), end="")
            self.print_tree(tree.right, indent + indent)
    
   
    
    def _make_prediction(self, x, tree):
        if tree.value is not None:
            # Ensure tree.value is an index into self.label_mapping.
            # If tree.value is already an original class label, this line will raise KeyError if the label doesn't exist in the mapping.
            return self.label_mapping.get(tree.value, tree.value)  # Provide a fallback if the key doesn't exist
        feature_val = x[tree.feature_index]
        if feature_val <= tree.threshold:
            return self._make_prediction(x, tree.left)
        else:
            return self._make_prediction(x, tree.right)

    def print_tree(self, tree=None, indent=" "):
        ''' function to print the tree '''     
        if not tree:
            tree = self.root
        if tree.value is not None:
            print(tree.value)
        else:
            print("X_"+str(tree.feature_index), "<=", tree.threshold, "?", tree.info_gain)
            print("%sleft:" % (indent), end="")
            self.print_tree(tree.left, indent + indent)
            print("%sright:" % (indent), end="")
            self.print_tree(tree.right, indent + indent)

    def prune(self, X_val, Y_val, epsilon=0.5):
        """
        Prune the decision tree using post-pruning method.
        """
        self._prune_node(self.root, X_val, Y_val, epsilon)



    def _prune_node(self, node, X_val, Y_val, epsilon):
        """
        Recursively prune the tree.
        """
        if node.left is not None or node.right is not None:
            # Prune child nodes first
            if node.left is not None:
                self._prune_node(node.left, X_val, Y_val, epsilon)
            if node.right is not None:
                self._prune_node(node.right, X_val, Y_val, epsilon)

            # After pruning children, check if current node can be pruned
            if node.left is not None and node.right is not None:
                if node.left.value is not None and node.right.value is not None:
                    # Calculate error before pruning
                    error_before = self._evaluate_error(X_val, Y_val)
                    
                    # Temporarily make current node a leaf
                    original_left, original_right = node.left, node.right
                    node.left, node.right = None, None
                    node.value = self.calculate_leaf_value(Y_val[node.feature_index <= node.threshold])

                    # Calculate error after pruning
                    error_after = self._evaluate_error(X_val, Y_val)

                    # Revert pruning if it does not reduce error
                    if error_before - error_after < epsilon:
                        node.left, node.right = original_left, original_right
                        node.value = None
    
    def _map_labels(self, Y):
        # Map labels to a continuous range of non-negative integers
        unique_labels, Y_mapped = np.unique(Y, return_inverse=True)
        self.label_mapping = {i: label for i, label in enumerate(unique_labels)}
        return self.label_mapping, Y_mapped.reshape(-1, 1)

## Purning method : 
For each node, the method determines if changing it into a leaf (by deleting its children) improves or preserves tree accuracy on a separate validation set (X_val, Y_val).This is found by comparing the error before and after trimming the node, as estimated in the _evaluate_error function. If the change in error is smaller than a modest threshold epsilon, suggesting that pruning does not have a large impact on the model's performance, the pruning is retained; the node's children are removed, transforming it into a leaf node with a class label defined by the majority class of the data points in it. In general it should reducing overfitting and improving its performance when dealing unseen data. 



In [ ]:
file_path = 'website-phishing.csv'
data = pd.read_csv(file_path, skiprows=1, header=None)
X = data.iloc[:, :-1].values
y = data.iloc[:, -1].values.reshape(-1, 1)

In [ ]:
# Split the data into training, validation, and testing sets
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.2, random_state=41)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.25, random_state=41)


# Initialize and fit the classifier
classifier = PurningDecisionTreeClassifier(min_samples_split=2, max_depth=5)
classifier.fit(X_train, y_train)

# Prune the tree
classifier.prune(X_val, y_val)
# Evaluate accuracy
y_pred = classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy after pruning: {accuracy:.4f}")

In [ ]:
#plotting
classifier.print_tree()

In [ ]:
file_path = 'BCP.csv'
data = pd.read_csv(file_path, skiprows=1, header=None)
X = data.iloc[:, 1:-1].values
y = data.iloc[:, -1].values.reshape(-1, 1)

In [ ]:
# Split the data into training, validation, and testing sets
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.2, random_state=41)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.25, random_state=41)


# Initialize and fit the classifier
classifier = PurningDecisionTreeClassifier(min_samples_split=2, max_depth=8)
classifier.fit(X_train, y_train)

# Prune the tree
classifier.prune(X_val, y_val)
# Evaluate accuracy
y_pred = classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy after pruning: {accuracy:.4f}")

In [ ]:
classifier.print_tree()

In [ ]:
file_path = 'new_arrhy.csv'
data = pd.read_csv(file_path, skiprows=1, header=None)
X = data.iloc[:, 1:-1].values
y = data.iloc[:, -1].values.reshape(-1, 1)
X = X.astype(np.int64)
y = y.astype(np.int64)

In [ ]:
# Split the data into training, validation, and testing sets
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.2, random_state=41)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.25, random_state=41)


# Initialize and fit the classifier
classifier = PurningDecisionTreeClassifier(min_samples_split=2, max_depth=5)
classifier.fit(X_train, y_train)

# Prune the tree
classifier.prune(X_val, y_val)
# Evaluate accuracy
y_pred = classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy after pruning: {accuracy:.4f}")

In [ ]:
classifier.print_tree()

After running tree code above that we can tell taht the purning slidy improved performance on web-phishing and BCP, but on the arrhythmia dataset, the accuracy rate drop by some reason, that could be following reasons: 
- Overfitting 
- Purning method cut relevant substree
- Dataset is too complexity 
  

In [ ]:
# loading website-phishing 
file_path = 'website-phishing.csv'
web = pd.read_csv(file_path, skiprows=1, header=None)
file_path = 'BCP.csv'
bcp = pd.read_csv(file_path, skiprows=1, header=None)
file_path = 'new_arrhy.csv'
arrhy = pd.read_csv(file_path, skiprows=1, header=None)

In [ ]:
X = web.iloc[:, 1:-1].values
y = web.iloc[:, -1].values.reshape(-1, 1)
X_train, X_test,y_train,y_test1 = train_test_split(X,y, test_size=0.2, random_state=41)
classifier1 = DecisionTreeClassifier(min_samples_split=2, max_depth=5)
classifier1.fit(X_train, y_train.ravel())  # Assuming y_train is a 2D array for sklearn compatibility
y_pred1 = classifier1.predict(X_test)
accuracy1 = accuracy_score(y_test1, y_pred1)

In [ ]:

# Split the data into training, validation, and testing sets
X_temp, X_test, y_temp, y_test2 = train_test_split(X, y, test_size=0.2, random_state=41)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.25, random_state=41)

classifier2 = PurningDecisionTreeClassifier(min_samples_split = 2, max_depth=5)
classifier2.fit(X_train, y_train)
classifier2.prune(X_val, y_val)
y_pred2 = classifier2.predict(X_test)

accuracy2 = accuracy_score(y_test2, y_pred2)
print(f"Accuracy of Decision Tree: {accuracy1}")
print(f"Accuracy of Decision Tree with purning: {accuracy2}")

In [ ]:
def calcuate_accuracy (y_true, y_pred):
    return np.mean(y_true == y_pred)


y_pred1 = np.array(y_pred1)
y_pred2 = np.array(y_pred2)
diff_accuracies = []
n_iterations = 1000
n_size = int(len(y_test1) * 0.50)  # Use 50% of the dataset for each bootstrap sample

for i in range(n_iterations):
    # Prepare train and test sets
    indices = resample(np.arange(len(y_test1)), n_samples=n_size)
    y_test_sample = y_test1[indices]
    y_pred1_sample = y_pred1[indices]
    y_pred2_sample = y_pred2[indices]
    
    # Calculate accuracy
    accuracy1 = calcuate_accuracy(y_test_sample, y_pred1_sample)
    accuracy2 = calcuate_accuracy(y_test_sample, y_pred2_sample)
    diff_accuracies.append(accuracy1 - accuracy2)

# Calculate p-value
diff_accuracies = np.array(diff_accuracies)
# Count how many times the difference in accuracies was at least as extreme as the observed difference
observed_diff = accuracy1 - accuracy2
p_value = np.sum(np.abs(diff_accuracies) >= np.abs(observed_diff)) / n_iterations

print(f'Estimated p-value: {p_value}')

In [ ]:
X = bcp.iloc[:, 1:-1].values
y = bcp.iloc[:, -1].values.reshape(-1, 1)
X_train, X_test,y_train,y_test1 = train_test_split(X,y, test_size=0.2, random_state=41)
classifier1 = DecisionTreeClassifier(min_samples_split=2, max_depth=5)
classifier1.fit(X_train, y_train.ravel())  # Assuming y_train is a 2D array for sklearn compatibility
y_pred1 = classifier1.predict(X_test)
accuracy1 = accuracy_score(y_test1, y_pred1)

In [ ]:

# Split the data into training, validation, and testing sets
X_temp, X_test, y_temp, y_test2 = train_test_split(X, y, test_size=0.2, random_state=41)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.25, random_state=41)

classifier2 = PurningDecisionTreeClassifier(min_samples_split = 2, max_depth=5)
classifier2.fit(X_train, y_train)
classifier2.prune(X_val, y_val)
y_pred2 = classifier2.predict(X_test)

accuracy2 = accuracy_score(y_test2, y_pred2)
print(f"Accuracy of Decision Tree: {accuracy1}")
print(f"Accuracy of Decision Tree with purning: {accuracy2}")

In [ ]:
def calcuate_accuracy (y_true, y_pred):
    return np.mean(y_true == y_pred)


y_pred1 = np.array(y_pred1)
y_pred2 = np.array(y_pred2)
diff_accuracies = []
n_iterations = 1000
n_size = int(len(y_test1) * 0.50)  # Use 50% of the dataset for each bootstrap sample

for i in range(n_iterations):
    # Prepare train and test sets
    indices = resample(np.arange(len(y_test1)), n_samples=n_size)
    y_test_sample = y_test1[indices]
    y_pred1_sample = y_pred1[indices]
    y_pred2_sample = y_pred2[indices]
    
    # Calculate accuracy
    accuracy1 = calcuate_accuracy(y_test_sample, y_pred1_sample)
    accuracy2 = calcuate_accuracy(y_test_sample, y_pred2_sample)
    diff_accuracies.append(accuracy1 - accuracy2)

# Calculate p-value
diff_accuracies = np.array(diff_accuracies)
# Count how many times the difference in accuracies was at least as extreme as the observed difference
observed_diff = accuracy1 - accuracy2
p_value = np.sum(np.abs(diff_accuracies) >= np.abs(observed_diff)) / n_iterations

print(f'Estimated p-value: {p_value}')

In [ ]:
X = arrhy.iloc[:, 1:-1].values
y = arrhy.iloc[:, -1].values.reshape(-1, 1)
X_train, X_test,y_train,y_test1 = train_test_split(X,y, test_size=0.2, random_state=41)
classifier1 = DecisionTreeClassifier(min_samples_split=2, max_depth=5)
classifier1.fit(X_train, y_train.ravel())  
y_pred1 = classifier1.predict(X_test)
accuracy1 = accuracy_score(y_test1, y_pred1)

In [ ]:

file_path = 'new_arrhy.csv'
data = pd.read_csv(file_path, skiprows=1, header=None)
X = data.iloc[:, 1:-1].values
y = data.iloc[:, -1].values.reshape(-1, 1)
X = X.astype(np.int64)
y = y.astype(np.int64)
# Split the data into training, validation, and testing sets
X_temp, X_test, y_temp, y_test2 = train_test_split(X, y, test_size=0.2, random_state=41)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.25, random_state=41)

classifier2 = PurningDecisionTreeClassifier(min_samples_split = 2, max_depth=5)
classifier2.fit(X_train, y_train)
classifier2.prune(X_val, y_val)
y_pred2 = classifier2.predict(X_test)

accuracy2 = accuracy_score(y_test2, y_pred2)
print(f"Accuracy of Decision Tree: {accuracy1}")
print(f"Accuracy of Decision Tree with purning: {accuracy2}")

In [ ]:
def calcuate_accuracy (y_true, y_pred):
    return np.mean(y_true == y_pred)


y_pred1 = np.array(y_pred1)
y_pred2 = np.array(y_pred2)
diff_accuracies = []
n_iterations = 1000
n_size = int(len(y_test1) * 0.50)  # Use 50% of the dataset for each bootstrap sample

for i in range(n_iterations):
    # Prepare train and test sets
    indices = resample(np.arange(len(y_test1)), n_samples=n_size)
    y_test_sample = y_test1[indices]
    y_pred1_sample = y_pred1[indices]
    y_pred2_sample = y_pred2[indices]
    
    # Calculate accuracy
    accuracy1 = calcuate_accuracy(y_test_sample, y_pred1_sample)
    accuracy2 = calcuate_accuracy(y_test_sample, y_pred2_sample)
    diff_accuracies.append(accuracy1 - accuracy2)

# Calculate p-value
diff_accuracies = np.array(diff_accuracies)
# Count how many times the difference in accuracies was at least as extreme as the observed difference
observed_diff = accuracy1 - accuracy2
p_value = np.sum(np.abs(diff_accuracies) >= np.abs(observed_diff)) / n_iterations

print(f'Estimated p-value: {p_value}')

After computing the p-values for comparisons of decision trees with and without pruning across three datasets, we discovered that they are consistently high. This shows that, in this scenario, the pruning strategy does not greatly improve decision tree performance.